# Cooking data

In [8]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import sqlite3
import json

Import the json training file.

In [24]:
with open('trainfile2.json', 'r') as json_file:
        training_data_no_format = json.load(json_file)

Setup database function.  Only needed to be run a single time for initial data setup.  Doesn't seem to create the database while jupyter is running, please use the setup_main.py if future use is needed.

In [27]:
categories = set()
foodies = sqlite3.connect('foodies.db')
foodies_cursor = foodies.cursor()
foodies_cursor.execute('''
    CREATE TABLE IF NOT EXISTS food(
    id int NOT NULL,
    category TEXT NOT NULL,
    UNIQUE(id));''')
foodies_cursor.execute('''
    CREATE TABLE IF NOT EXISTS ingredients(
    id integer PRIMARY KEY AUTOINCREMENT,
    ingredient TEXT NOT NULL,
    UNIQUE(ingredient));''')
foodies_cursor.execute('''
    CREATE TABLE IF NOT EXISTS bridge(
    food_id integer NOT NULL REFERENCES food(id),
    ingredients_id integer NOT NULL REFERENCES ingredients(id),
    PRIMARY KEY (food_id, ingredients_id));''')
foodies.commit()
for item in training_data_no_format:
    categories.add(item['cuisine'])
    foodies_cursor.execute('''
        INSERT OR IGNORE INTO food(id, category)
        VALUES(?,?)''', [item['id'], item['cuisine']])
    item_id = foodies_cursor.lastrowid
    for ingredient in item['ingredients']:
        foodies_cursor.execute('''
            INSERT OR IGNORE INTO ingredients(id, ingredient)
            VALUES(?,?);''', [None, ingredient])
        ingredient_id = foodies_cursor.lastrowid
        foodies_cursor.execute('''
            INSERT OR IGNORE INTO bridge(food_id, ingredients_id)
            VALUES(?,?);''', [item_id, ingredient_id])
foodies.commit()

In [28]:
print(f'The number of categories is {len(categories)}.')
print(f'{categories}')

The number of categories is 20.
{'moroccan', 'spanish', 'vietnamese', 'chinese', 'italian', 'korean', 'greek', 'french', 'brazilian', 'jamaican', 'cajun_creole', 'thai', 'filipino', 'british', 'japanese', 'russian', 'southern_us', 'mexican', 'indian', 'irish'}
